In [11]:
import math
import random


p = 2**127 - 1      # Mersenne
F = lambda x: x % p

'''
witness: x
output: x^2 = 25

We should proof that we know x as x^2 = 25
'''

'\nwitness: x\noutput: x^2 = 25\n\nWe should proof that we know x as x^2 = 25\n'

In [12]:
# I. Arithmetization (R1CS Rank 1 Constraint System)
# witness: x; vector = [1, x, x^2]
# R1CS build : <A_i, w> * <B_i, w> = <C_i, w>

# Constraint 1: x*x = x^2
# Constraint 1: x^2 = 25

A = [
    [0, 1, 0],
    [0, 0, 1]
]
B = [
    [0, 1, 0],
    [1, 0, 0]
]
C = [
    [0, 0, 1],
    [0, 0, 0]
]

K = [0, 25]

N_CONSTRAINTS = 2
N_VARS = 3

print("R1CS ")
print("- 2 constraints")
print("- 3 vars (1, x, x^2) -> vector")
print("A:", A)
print("B:", B)
print("C:", C)
print("K:", K)



R1CS 
- 2 constraints
- 3 vars (1, x, x^2) -> vector
A: [[0, 1, 0], [0, 0, 1]]
B: [[0, 1, 0], [1, 0, 0]]
C: [[0, 0, 1], [0, 0, 0]]
K: [0, 25]


In [13]:
def check_r1cs(x, nb_constraint=2, nb_var=3):
    w = [1, x, x*x]

    for i in range(nb_constraint):
        A_dot = sum(A[i][j] * w[j] for j in range(nb_var))
        B_dot = sum(B[i][j] * w[j] for j in range(nb_var))
        C_dot = sum(C[i][j] * w[j] for j in range(nb_var)) + K[i]

        print(f"Constraint {i+1}:  {A_dot} * {B_dot} =? {C_dot}")

        if A_dot * B_dot != C_dot:
            print("R1CS invalid")
            return False
    
    print("R1CS valid")
    return True

check_r1cs(4, N_CONSTRAINTS, N_VARS)
print("-----")
check_r1cs(5, N_CONSTRAINTS, N_VARS)
print("-----")
check_r1cs(7, N_CONSTRAINTS, N_VARS)

Constraint 1:  4 * 4 =? 16
Constraint 2:  16 * 1 =? 25
R1CS invalid
-----
Constraint 1:  5 * 5 =? 25
Constraint 2:  25 * 1 =? 25
R1CS valid
-----
Constraint 1:  7 * 7 =? 49
Constraint 2:  49 * 1 =? 25
R1CS invalid


False

In [20]:
# Constraints to polynomial ( R1CS -> QAP Quadratic Arithmetic Program) 
# Transform linear constraints to a unique polynomial, verifiable with an single secret point

def lagrange_basis(i, n):
    def poly(t): #closure func
        num = 1
        den = 1
        for j in range(1, n+1):
            if j != i:
                num = F(num * (t-j))
                den = F(den * (i-j))
        return F(num * pow(den, -1, p))
    return poly

L = [lagrange_basis(i, N_CONSTRAINTS) for i in range(1, N_CONSTRAINTS+1)]

print("Constructed basic Lagrange polynomials.")

def poly_from_R1CS_vector(vec):
    """Build a polynomial (N_CONSTRAINTS-1) from L_i."""
    def poly(t):
        acc = 0
        for i in range(n_constraints):
            acc = F(acc + vec[i] * L[i](t))
        return acc
    return poly

A_poly = [poly_from_R1CS_vector([A[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
B_poly = [poly_from_R1CS_vector([B[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
C_poly = [poly_from_R1CS_vector([C[i][j] for i in range(N_CONSTRAINTS)]) for j in range(N_VARS)]
K_poly = poly_from_R1CS_vector(K)

print("Polynomials A(t), B(t), C(t), K(t) generated."

Constructed basic Lagrange polynomials.
170141183460469231731687303715884105724


In [ ]:
# Parameters

# Proof gen

# Verification